In [310]:
import xarray as xr
import numpy as np
import sys
import glob
import matplotlib.pyplot as plt
import pandas as pd
import dask

In [311]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '30GB',
    processes = 1,
    queue = 'casper',
    local_directory = '$TMPDIR',
    resource_spec = 'select=1:ncpus=1:mem=30GB',
    project='P04010022',
    walltime='01:00:00',
    interface='mgt')

# scale up
cluster.scale(12)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/u/home/islas/miniconda3/envs/islaenv24/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37485 instead
  f"Port {expected} is already in use.\n"


In [314]:
cluster

PBSCluster(6a6f628e, 'tcp://10.18.206.66:38833', workers=12, threads=12, memory=335.28 GiB)

In [315]:
client

<Client: 'tcp://10.18.206.66:38833' processes=12 threads=12, memory=335.28 GiB>

### Information on start, end year and initialization month

In [342]:
ystart=1970 ; yend=2020
nyears=yend-ystart+1
initmon=2
initmonstr=str(initmon).zfill(2)
domain="atm" ; freq="month_1" ; var="PSL"
pathout="/glade/campaign/cgd/cas/islas/python_savs/NCAR_CW3E_SMYLE/DATA_SORT/NAO/"

### NAO station index longitudes and latitudes

In [343]:
reyklon = 360.-21.9426 ; reyklat = 64.1466
azorelon = 360.-25.6687 ; azorelat = 37.7412

### Subroutine for fixing the time axis using time_bnds

In [344]:
def fixtime(dat):
    diff = np.array(dat.time_bnds.isel(nbnd=1, M=0)) - np.array(dat.time_bnds.isel(nbnd=0, M=0))
    diff = diff/2.
    newtime = np.array(dat.time_bnds.isel(nbnd=0, M=0)) + diff
    dat['time'] = newtime
    return dat

### L32

In [345]:
basepath="/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/"
expname="b.e21.BSMYLE-CW3E.f09_g17"
nmems=20
memstr=[str(i).zfill(3) for i in np.arange(1,nmems+1,1)]

In [346]:
all_reyk=[]
all_az=[]
times=[]
for iyear in np.arange(ystart,yend+1,1):
    filelist = sorted([  glob.glob(basepath+expname+"."+str(iyear)+'-'+str(initmon).zfill(2)+'.'+imem+'/atm/proc/tseries/month_1/*.PSL.*')[0] for imem in memstr ])
    dat = xr.open_mfdataset(filelist, concat_dim='M', combine='nested')
    dat = fixtime(dat)
    dat = dat.PSL
    times.append(dat.time)
    year = dat.time.dt.year ; month = dat.time.dt.month ; day = dat.time.dt.day
    yearout = xr.where( year > year[0], 1971, 1970 )
    dat['time'] = [ pd.Timestamp(yearout[i].item(), month[i].item(), day[i].item()) for i in np.arange(0,dat.time.size,1) ]
    reyk = dat.sel(lon = reyklon, lat=reyklat, method='nearest').load()
    az = dat.sel(lon = azorelon, lat=azorelat, method='nearest').load()

    #---A couple of members are corrupted so I'm replacing them with other members for now to get the code to work
    if initmon == 2:
        if (iyear == 1971):
            reyk[6,:] = reyk[5,:]
            az[6,:] = az[5,:]
        if (iyear == 1975):
            reyk[3,:] = reyk[2,:]
            az[3,:] = az[2,:]
    
    all_reyk.append(reyk)
    all_az.append(az)

all_reyk = xr.concat(all_reyk, dim='init_year')
all_az = xr.concat(all_az, dim='init_year')
#times = xr.concat(times, dim='init_year')
all_reyk['init_year'] = np.arange(ystart,yend+1,1)
all_az['init_year'] = np.arange(ystart,yend+1,1)
#times['init_year'] = np.arange(ystart,yend+1,1)

In [347]:
all_reyk = (all_reyk - all_reyk.mean('init_year')) / all_reyk.std('init_year')
all_az = (all_az - all_az.mean('init_year')) / all_az.std('init_year')
nao_l32 = all_az - all_reyk
nao_l32 = nao_l32.rename('NAO')
nao_l32.to_netcdf(pathout+'NAO_stationbased_L32_initmon'+initmonstr+'.nc')

### L83

In [348]:
basepath="/glade/campaign/cesm/development/espwg/SMYLE-CW3E-L83/timeseries/"
expname="b.e21.BSMYLE-CW3E-L83.f09_g17"
nmems=20
memstr=[str(i).zfill(3) for i in np.arange(1,nmems+1,1)]

In [349]:
all_reyk=[]
all_az=[]
for iyear in np.arange(ystart,yend+1,1):
    filelist = sorted([  glob.glob(basepath+expname+"."+str(iyear)+'-'+str(initmon).zfill(2)+'.'+imem+'/atm/proc/tseries/month_1/*.PSL.*')[0] for imem in memstr ])
    dat = xr.open_mfdataset(filelist, concat_dim='M', combine='nested')
    dat = fixtime(dat)
    dat = dat.PSL
    year = dat.time.dt.year ; month = dat.time.dt.month ; day = dat.time.dt.day
    yearout = xr.where( year > year[0], 1971, 1970 )
    dat['time'] = [ pd.Timestamp(yearout[i].item(), month[i].item(), day[i].item()) for i in np.arange(0,dat.time.size,1) ]
    reyk = dat.sel(lon = reyklon, lat=reyklat, method='nearest').load()
    az = dat.sel(lon = azorelon, lat=azorelat, method='nearest').load()
    all_reyk.append(reyk)
    all_az.append(az)
all_reyk = xr.concat(all_reyk, dim='init_year')
all_az = xr.concat(all_az, dim='init_year')
all_reyk['init_year'] = np.arange(ystart,yend+1,1)
all_az['init_year'] = np.arange(ystart,yend+1,1)

In [350]:
all_reyk = (all_reyk - all_reyk.mean('init_year')) / all_reyk.std('init_year')
all_az = (all_az - all_az.mean('init_year')) / all_az.std('init_year')
nao_l83 = all_az - all_reyk
nao_l83 = nao_l83.rename('NAO')
nao_l83.to_netcdf(pathout+'NAO_stationbased_L83_initmon'+initmonstr+'.nc')

### ERA5

In [351]:
era5 = xr.open_mfdataset("/glade/campaign/cgd/cas/observations/ERA5/mon_f09/PSL/*.nc")
all_reyk=[]
all_az=[]
for iyear in np.arange(ystart,yend+1,1):
    thistime = times[iyear-ystart].values
    thisyear  = [ era5.sel(time=pd.Timestamp(i.year,i.month,1), method='nearest') for i in thistime ]
    dat = xr.concat(thisyear, dim='time')

    year = dat.time.dt.year ; month = dat.time.dt.month ; day = dat.time.dt.day
    #yearout = xr.where( year > year[0], 1971, 1970 )
    #dat['time'] = [ pd.Timestamp(yearout[i].item(), month[i].item(), day[i].item()) for i in np.arange(0,dat.time.size,1) ]
    dat['time'] = nao_l32.time
    dat = dat.load()
    reyk = dat.PSL.sel(lon = reyklon, lat=reyklat, method='nearest').load()
    az = dat.PSL.sel(lon = azorelon, lat=azorelat, method='nearest').load()

    all_reyk.append(reyk)
    all_az.append(az)
    del(thisyear)

all_reyk = xr.concat(all_reyk, dim='init_year')
all_az = xr.concat(all_az, dim='init_year')
all_reyk['init_year'] = np.arange(ystart,yend+1,1)
all_az['init_year'] = np.arange(ystart,yend+1,1)

In [352]:
all_reyk = (all_reyk - all_reyk.mean('init_year')) / all_reyk.std('init_year')
all_az = (all_az - all_az.mean('init_year')) / all_az.std('init_year')
nao_era5 = all_az - all_reyk
nao_era5 = nao_era5.rename('NAO')
nao_era5.to_netcdf(pathout+'NAO_stationbased_ERA5_initmon'+initmonstr+'.nc')

### JRA3Q

In [353]:
jra3q = xr.open_mfdataset("/glade/campaign/cgd/cas/observations/JRA3Q/mon/psl/*.nc")
all_reyk = []
all_az = []
for iyear in np.arange(ystart,yend+1,1):
    thistime = times[iyear-ystart].values
    thisyear  = [ jra3q.sel(time=pd.Timestamp(i.year,i.month, i.day), method='nearest') for i in thistime ]
    dat = xr.concat(thisyear, dim='time')

    year = dat.time.dt.year ; month = dat.time.dt.month ; day = dat.time.dt.day
    dat['time'] = nao_l32.time
    dat = dat.load()
    reyk = dat.psl.sel(lon = reyklon, lat=reyklat, method='nearest').load()
    az = dat.psl.sel(lon = azorelon, lat=azorelat, method='nearest').load()

    all_reyk.append(reyk)
    all_az.append(az)
    del(thisyear)

all_reyk = xr.concat(all_reyk, dim='init_year')
all_az = xr.concat(all_az, dim='init_year')
all_reyk['init_year'] = np.arange(ystart,yend+1,1)
all_az['init_year'] = np.arange(ystart,yend+1,1)

In [354]:
all_reyk = (all_reyk - all_reyk.mean('init_year')) / all_reyk.std('init_year')
all_az = (all_az - all_az.mean('init_year')) / all_az.std('init_year')
nao_jra3q = all_az - all_reyk
nao_jra3q = nao_jra3q.rename('NAO')
nao_jra3q.to_netcdf(pathout+'NAO_stationbased_JRA3Q_initmon'+initmonstr+'.nc')

In [355]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
